# Data Mining (KEN4113) #
## Lab 3: Classification ##

Konstantin Sandfort
i6255681

In [80]:
# Imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.pipeline import make_pipeline
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif
from sklearn.feature_selection import chi2
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import VarianceThreshold
from pandas import CategoricalDtype

## Step 1: EDA
The EDA will provide better insights of the data and give us a better idea of the variables that can be used for classification.

In [108]:
# Import data
data_train = pd.read_csv('caravan.csv')
data_test = pd.read_csv('caravanTest.csv')
y = data_train['CARAVAN POLICY']
pd.set_option('display.max_columns', None)
print(data_test.head(10))

   Customer Subtype  Number of houses  Avg size household  Avg Age  \
0                33                 1                   4        2   
1                 6                 1                   3        2   
2                39                 1                   3        3   
3                 9                 1                   2        3   
4                31                 1                   2        4   
5                30                 1                   2        4   
6                35                 1                   2        4   
7                 6                 1                   3        3   
8                 4                 1                   2        4   
9                10                 1                   4        2   

   Customer main type  Roman catholic  Protestant  Other religion  \
0                   8               0           6               0   
1                   2               0           5               0   
2                   9 

In [111]:
# --- Step 1: Preprocessing ---

# One-hot-encoding of l0 and l2

def ohe(df, col_name, label):
    cols = pd.get_dummies(df,
                          prefix=label,
                          dtype=int,
                          columns=[col_name],
                          drop_first=False)
    return cols

data_train_ohe = ohe(data_train, 'Customer Subtype', 'l0')
data_train_ohe = ohe(data_train_ohe, 'Customer main type', 'l2')

print(data_train_ohe.head(10))

   Number of houses  Avg size household  Avg Age  Roman catholic  Protestant  \
0                 1                   3        2               0           5   
1                 1                   2        2               1           4   
2                 1                   2        2               0           4   
3                 1                   3        3               2           3   
4                 1                   4        2               1           4   
5                 1                   2        1               0           5   
6                 2                   3        2               2           2   
7                 1                   2        3               0           7   
8                 1                   2        4               0           1   
9                 2                   3        3               3           5   

   Other religion  No religion  Married  Living together  Other relation  \
0               1            3        7    

In [113]:
# --- Step 2: Feature Selection
# a) Removing variables with low variance

def variance_threshold_keep_col_names(df, thr):
    var_threshold = VarianceThreshold(thr)
    var_threshold.fit_transform(df)
    return df[df.columns[var_threshold.get_support(indices=True)]]

data_train_new = variance_threshold_keep_col_names(data_train_ohe, 0.8 * (1 - 0.8))

#print(new_cols)
print(data_train_new.columns, 'Length', len(data_train_new.columns))

Index(['Number of houses', 'Avg size household', 'Avg Age', 'Roman catholic',
       'Protestant', 'Other religion', 'No religion', 'Married',
       'Living together', 'Other relation', 'Singles',
       'Household without children', 'Household with children',
       'High level education', 'Medium level education',
       'Lower level education', 'High status', 'Entrepreneur', 'Farmer',
       'Middle Management', 'Skilled labourers', 'Unskilled labourers',
       'Social class A', 'Social class B1', 'Social class B2',
       'Social class C', 'Social class D', 'Rented house', 'Home owners',
       '1 car', '2 cars', 'No car', 'National Health Service',
       'Private health insurance', 'Income < 30.000', 'Income 30-45.000',
       'Income 45-75.000', 'Income 75-122.000', 'Income >123.000',
       'Average income', 'Purchasing power class',
       'Contribution private third party insurance',
       'Contribution third party insurance (agriculture)',
       'Contribution car policie

In [54]:
def univariate_feature_selection(df, y, k_f):
    df_new = SelectKBest(f_classif, k=k_f).fit_transform(df, y)
    return df_new